# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,lscc_Database_ID,luad_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602,0.988651,0.833013,0.886406,0.899548,0.293107,0.856468,-0.102606,-0.119042,-0.037599,-0.056728,-0.442692,-0.294055
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005.2,NP_000005,0.977133,0.869751,0.797420,0.720246,0.432296,0.963139,0.331652,-0.081407,-0.094712,-0.198597,-0.461877,-0.244195
A2ML1,NP_653271.2|NP_001269353.1,NaN,NP_653271.2|NP_001269353.1,NP_653271,0.983618,NaN,0.995824,0.682597,NaN,0.997940,-0.303609,NaN,0.005690,0.732705,NaN,0.052487
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.983618,0.672177,0.904402,0.162286,0.317790,0.997310,0.049927,-0.100606,0.020226,0.199360,0.219718,0.030730
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.990196,0.700758,0.916913,0.933806,0.921447,0.979319,0.109607,0.089959,0.039000,0.053219,-0.103594,0.132034
AADAT,NaN,NP_001273611.1|NP_001273612.1,NP_057312.1|NP_001273611.1,NaN,0.977133,NaN,NaN,NaN,0.903755,NaN,-0.569013,NaN,-0.399648,NaN,-0.254802,NaN
AAED1,NP_714542.1,NP_714542.1,NP_714542.1,NaN,0.983618,NaN,0.839344,0.835022,0.672262,NaN,0.135681,NaN,0.101844,0.098621,-0.273658,NaN
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258815.1,NP_078942,0.918695,0.756983,0.925489,0.772871,0.774339,0.997310,0.367299,-0.078345,-0.026190,0.087950,-0.119763,0.049414
AAK1,NP_055726.3,NaN,NP_055726.3,NP_055726,0.994544,0.907166,0.955870,0.364152,NaN,0.964182,0.020368,0.025289,-0.011070,-0.192113,NaN,0.104969


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.988651,-0.102606
1,brca,A2M,NP_000005.2,0.977133,0.331652
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.983618,-0.303609
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.983618,0.049927
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.990196,0.109607
5,brca,AADAT,NP_057312.1|NP_001273611.1,0.977133,-0.569013
6,brca,AAED1,NP_714542.1,0.983618,0.135681
7,brca,AAGAB,NP_078942.3|NP_001258815.1,0.918695,0.367299
8,brca,AAK1,NP_055726.3,0.994544,0.020368
9,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.994907,-0.056287


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,CNOT8,NP_001288002.1|NP_001288003.1|NP_001288006.1|N...,0.000003,1.218408,1.0
1,colon,ATP6V1A,NaN,0.007309,-0.220323,-1.0
2,colon,ATP6V1E1,NaN,0.003844,-0.250746,-1.0
3,colon,ATP6V1G1,NaN,0.006237,-0.280398,-1.0
4,colon,ATP6V1H,NaN,0.003844,-0.235647,-1.0
5,colon,CRELD2,NaN,0.007309,-0.494643,-1.0
6,colon,DNAJC19,NaN,0.039001,0.333038,1.0
7,colon,ERP29,NaN,0.041833,-0.250365,-1.0
8,colon,NDUFAF2,NaN,0.024441,0.347422,1.0
9,colon,NUCB2,NaN,0.010832,-0.426657,-1.0


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
CNOT8,"[brca, lscc, luad]",1.0,3
ATP6V1E1,"[colon, lscc, luad]",-1.0,3
ATP6V1H,"[colon, lscc, luad]",-1.0,3
ATP6V1G1,"[colon, lscc]",-1.0,2
CRELD2,"[colon, lscc]",-1.0,2
DNAJC19,"[colon, lscc]",1.0,2
ATP6V1A,"[colon, luad]",-1.0,2
CYP7B1,"[hnscc, lscc]",-1.0,2
SMAP2,"[hnscc, lscc]",-1.0,2


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3      3
2     11
1    860
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")